In [8]:
# Gender Pronoun Transformation System
# Install and import required packages
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy
import pandas as pd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -------------------- ------------------- 6.6/12.8 MB 33.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 32.9 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 29.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
class PronounGenderTransformer:
    """
    A rules-based system for transforming gendered pronouns in sentences.
    Transforms ALL gendered pronouns to the target gender with proper grammar.
    """
    
    def __init__(self):
        # Define pronoun transformations for each target gender
        self.to_female_map = {
            # Male to Female
            'he': 'she', 'He': 'She',
            'him': 'her', 'Him': 'Her', 
            'his': 'her', 'His': 'Her',  # Possessive his -> her
            'himself': 'herself', 'Himself': 'Herself',
            
            # Female stays Female (no change needed)
            'she': 'she', 'She': 'She',
            'her': 'her', 'Her': 'Her',
            'hers': 'hers', 'Hers': 'Hers',
            'herself': 'herself', 'Herself': 'Herself'
        }
        
        self.to_male_map = {
            # Female to Male  
            'she': 'he', 'She': 'He',
            'her': None,  # Context dependent - will handle separately
            'Her': None,  # Context dependent - will handle separately
            'hers': 'his', 'Hers': 'His',
            'herself': 'himself', 'Herself': 'Himself',
            
            # Male stays Male (no change needed)
            'he': 'he', 'He': 'He',
            'him': 'him', 'Him': 'Him',
            'his': 'his', 'His': 'His', 
            'himself': 'himself', 'Himself': 'Himself'
        }
    
    def transform_her_to_male(self, token, doc):
        """Handle the context-dependent transformation of 'her' to male pronouns."""
        is_capitalized = token.text[0].isupper()
        
        # Check grammatical role using dependency parsing
        if token.dep_ == 'poss':  # Possessive: "her book" -> "his book"
            return 'His' if is_capitalized else 'his'
        else:  # Objective: "give her" -> "give him"
            return 'Him' if is_capitalized else 'him'
    
    def transform_sentence(self, sentence, target_gender):
        """
        Transform all gendered pronouns in a sentence to target gender.
        
        Args:
            sentence: Input sentence string
            target_gender: 'male' or 'female'
            
        Returns:
            Transformed sentence string
        """
        doc = nlp(sentence)
        transformed_tokens = []
        
        # Choose the appropriate mapping
        if target_gender.lower() == 'female':
            pronoun_map = self.to_female_map
        elif target_gender.lower() == 'male':
            pronoun_map = self.to_male_map
        else:
            raise ValueError("Target gender must be 'male' or 'female'")
        
        for token in doc:
            token_text = token.text
            
            # Check if this is a gendered pronoun we need to transform
            if token_text in pronoun_map:
                if token_text in ['her', 'Her'] and target_gender.lower() == 'male':
                    # Special handling for context-dependent 'her'
                    transformed = self.transform_her_to_male(token, doc)
                else:
                    # Standard transformation
                    transformed = pronoun_map[token_text]
                
                transformed_tokens.append(transformed)
            else:
                transformed_tokens.append(token_text)
        
        # Reconstruct the sentence with proper spacing
        result = ""
        for i, token in enumerate(doc):
            if i == 0:
                result += transformed_tokens[i]
            else:
                if not token.is_punct:
                    result += " " + transformed_tokens[i]
                else:
                    result += transformed_tokens[i]
        
        return result

# Create the transformer instance
transformer = PronounGenderTransformer()

print("PRONOUN GENDER TRANSFORMATION SYSTEM")
print("=" * 50)
print("Handles all gendered pronouns correctly")
print("Preserves grammatical correctness") 
print("Context-aware transformation of 'her'")
print("Maintains proper capitalization")
print()

# Test the exact example from the requirements
print("EXAMPLE FROM REQUIREMENTS:")
example_input = "He gave her his book."
example_target = "female"
example_output = transformer.transform_sentence(example_input, example_target)
print(f"Input: '{example_input}'")
print(f"Target gender: {example_target}")  
print(f"Output: '{example_output}'")
print(f"Expected: 'She gave her her book.'")
print(f"Match: {example_output == 'She gave her her book.'}")
print()

# Basic test cases
test_cases = [
    ("He gave her his book.", "female", "She gave her her book."),
    ("He gave her his book.", "male", "He gave him his book."),
    ("He thinks about himself.", "female", "She thinks about herself."),
    ("She bought herself a gift.", "male", "He bought himself a gift."),
    ("This is her book.", "male", "This is his book."),
    ("Give the book to her.", "male", "Give the book to him."),
    ("The book is hers.", "male", "The book is his."),
    ("The car is his.", "female", "The car is her."),
]

print("TEST CASES:")
print("-" * 30)
for i, (input_text, target, expected) in enumerate(test_cases, 1):
    result = transformer.transform_sentence(input_text, target)
    passed = result == expected
    
    status = "PASS" if passed else "FAIL"
    print(f"{status} Test {i:2d}: '{input_text}' -> {target}")
    print(f"         Result: '{result}'")
    if not passed:
        print(f"         Expected: '{expected}'")
    print()

print("SYSTEM IMPLEMENTATION COMPLETE")

PRONOUN GENDER TRANSFORMATION SYSTEM
Handles all gendered pronouns correctly
Preserves grammatical correctness
Context-aware transformation of 'her'
Maintains proper capitalization

EXAMPLE FROM REQUIREMENTS:
Input: 'He gave her his book.'
Target gender: female
Output: 'She gave her her book.'
Expected: 'She gave her her book.'
Match: True

TEST CASES:
------------------------------
PASS Test  1: 'He gave her his book.' -> female
         Result: 'She gave her her book.'

PASS Test  2: 'He gave her his book.' -> male
         Result: 'He gave him his book.'

PASS Test  3: 'He thinks about himself.' -> female
         Result: 'She thinks about herself.'

PASS Test  4: 'She bought herself a gift.' -> male
         Result: 'He bought himself a gift.'

PASS Test  5: 'This is her book.' -> male
         Result: 'This is his book.'

PASS Test  6: 'Give the book to her.' -> male
         Result: 'Give the book to him.'

PASS Test  7: 'The book is hers.' -> male
         Result: 'The book is h

In [10]:
# Test against provided test cases
print("TESTING AGAINST PROVIDED TEST CASES")
print("=" * 50)

# Load test cases from CSV
try:
    test_df = pd.read_csv('pronoun_testcases.csv')
    print(f"Loaded {len(test_df)} test cases from 'pronoun_testcases.csv'")
    print()
    
    # Run tests
    correct = 0
    total = len(test_df)
    
    print("RUNNING VALIDATION TESTS:")
    print("-" * 30)
    
    for idx, row in test_df.iterrows():
        input_text = row['input_text']
        target_gender = row['target_gender'] 
        expected_output = row['expected_output']
        
        # Get result from our system
        actual_output = transformer.transform_sentence(input_text, target_gender)
        
        # Check if correct
        is_correct = actual_output == expected_output
        if is_correct:
            correct += 1
            
        # Print result
        status = "PASS" if is_correct else "FAIL"
        print(f"{status} Test {idx+1:2d}: '{input_text}' -> {target_gender}")
        print(f"    Expected: '{expected_output}'")
        print(f"    Got:      '{actual_output}'")
        if not is_correct:
            print(f"    MISMATCH!")
        print()
    
    # Final results
    accuracy = (correct / total) * 100
    print("=" * 50)
    print(f"FINAL RESULTS:")
    print(f"   Correct: {correct}/{total}")
    print(f"   Accuracy: {accuracy:.1f}%")
    print(f"   Status: {'PASSED' if accuracy == 100 else 'NEEDS IMPROVEMENT'}")
    
    if accuracy == 100:
        print()
        print("CONGRATULATIONS! All test cases passed!")
        print("The pronoun gender transformation system is working perfectly!")
    else:
        print(f)
        print(f"{total - correct} test case(s) failed. Review the implementation.")

except FileNotFoundError:
    print("ERROR: Test file 'pronoun_testcases.csv' not found in current directory")
except Exception as e:
    print(f"ERROR: Error loading test cases: {e}")

print()
print("=" * 70)
print("PRONOUN GENDER TRANSFORMATION SYSTEM - IMPLEMENTATION COMPLETE")
print("=" * 70)

TESTING AGAINST PROVIDED TEST CASES
Loaded 26 test cases from 'pronoun_testcases.csv'

RUNNING VALIDATION TESTS:
------------------------------
PASS Test  1: 'He is going to the market.' -> female
    Expected: 'She is going to the market.'
    Got:      'She is going to the market.'

PASS Test  2: 'His book is on the table.' -> female
    Expected: 'Her book is on the table.'
    Got:      'Her book is on the table.'

PASS Test  3: 'I saw him yesterday.' -> female
    Expected: 'I saw her yesterday.'
    Got:      'I saw her yesterday.'

PASS Test  4: 'He hurt himself.' -> female
    Expected: 'She hurt herself.'
    Got:      'She hurt herself.'

PASS Test  5: 'I called him last night.' -> female
    Expected: 'I called her last night.'
    Got:      'I called her last night.'

PASS Test  6: 'That is his car.' -> female
    Expected: 'That is her car.'
    Got:      'That is her car.'

PASS Test  7: 'He told me about his trip.' -> female
    Expected: 'She told me about her trip.'
  